## Ingestion

In [29]:
import pandas as pd

In [30]:
df = pd.read_csv('../data/fitness_exercises_500.csv')

In [31]:
df.head()

,id,exercise_name,type_of_activity,type_of_equipment,body_part,type,muscle_groups_activated,instruction
0,0,Push-Up Hold,mobility,dip belt,lower,stretch,"glutes, quads, hamstrings",Setup: Prepare equipment: dip belt. Ensure sta...
1,1,Explosive Lateral Raise Pulse,cardio,barbells,full body,hold,"back, chest, legs",Setup: Set barbell with collars secured; grip ...
2,2,Rotational Jumping Jack Iso,warm-up,barbells,upper,push,"biceps, forearms, chest",Setup: Set barbell with collars secured; grip ...
3,3,Wide-Grip Running,strength,kettlebells,upper,pull,"deltoids, triceps, forearms",Setup: Place kettlebell close to midline; hing...
4,4,Decline Dips,cardio,barbells,core,stretch,"abs, lower back, obliques",Setup: Set barbell with collars secured; grip ...


In [32]:
import minsearch

In [33]:
df.columns

Index(['id', 'exercise_name', 'type_of_activity', 'type_of_equipment',
       'body_part', 'type', 'muscle_groups_activated', 'instruction'],
      dtype='object')

In [34]:
documents = df.to_dict(orient='records')

In [35]:
index = minsearch.Index(
    text_fields=['exercise_name', 'type_of_activity', 'type_of_equipment', 'body_part',
       'type', 'muscle_groups_activated', 'instruction'],
    keyword_fields=['id']
)

In [36]:
index.fit(documents)

In [37]:
query = 'give me leg exercises for hamstrings'

## Implementing RAG Flow

In [38]:
from google import genai

In [39]:
client = genai.Client()

In [40]:
def search(query):
    boost = {}

    results = index.search(
        query=query,
        boost_dict=boost,
        filter_dict={},
        num_results=10
    )

    return results

In [41]:
documents[0]

{'id': 0,
 'exercise_name': 'Push-Up Hold',
 'type_of_activity': 'mobility',
 'type_of_equipment': 'dip belt',
 'body_part': 'lower',
 'type': 'stretch',
 'muscle_groups_activated': 'glutes, quads, hamstrings',
 'instruction': 'Setup: Prepare equipment: dip belt. Ensure stable setup and safe load. Brace core, keep neutral spine, drive through whole foot. Primary focus: glutes, quads, hamstrings. Movement: Gently enter the stretch until a mild pull is felt; hold without bouncing and keep breathing. Tempo: 2‑2‑2‑0. Dose: 1–2 sets × 30–60 sec each side. Rest 45–90s between sets. Breathing: Slow nasal breaths; exhale to deepen slightly, no pain. Coaching cues: keep neck long, ribs down, and knees tracking over toes. Common mistakes: avoid bouncing; stay within mild discomfort. Do not bounce or hyperextend joints. Safety: Never force end range; joints should feel stable.'}

In [42]:
#We want the LLM to put the document content in the context of the answer
prompt_template = """
    You're a professional fitness assistant. Answer the QUESTION based only on the CONTEXT provided from the exercise & fitness database.  
    
    - Use only the facts from the CONTEXT when answering the QUESTION.  
    - If the CONTEXT does not contain the answer, respond with: NONE.  
    - Keep your answer clear, concise, and detail with instruction for fitness use.  
    
    QUESTION: {question}  
    
    CONTEXT:  
    {context}  
""".strip()

entry_template = """
exercise_name: {exercise_name}',
type_of_activity: {type_of_activity},
type_of_equipment: {type_of_equipment},
body_part: {body_part},
type: {type},
muscle_groups_activated: {muscle_groups_activated},
instruction: {instruction}
""".strip()

def build_prompt(query, search_results):    
    context = ""
    
    for doc in search_results:
        context = context + entry_template.format(**doc) + "\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [43]:
def llm(prompt):
    response = client.models.generate_content(
    model="gemini-2.5-flash",
    contents=prompt
    )

    return response.text

In [44]:
def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [45]:
query = 'I want some core exercises that also help my back'

answer = rag(query)
print(answer)

ServerError: 503 UNAVAILABLE. {'error': {'code': 503, 'message': 'The service is currently unavailable.', 'status': 'UNAVAILABLE'}}